In [ ]:
import abc

## Multiple Inheritance in Python 2
* Python's Old Style classes


In [150]:
class A: 
    def __init__(self): 
        print('A.__init__')
class B(A): 
    def __init__(self):  
        A.__init__(self)
        print('B.__init__')
class C(A): 
    def __init__(self): 
        A.__init__(self)
        print('C.__init__')
class D(B,C): 
    def __init__(self):
        B.__init__(self)
        C.__init__(self)
        print('D.__init__')
D()

A.__init__
B.__init__
A.__init__
C.__init__
D.__init__


## Multiple Inheritance in Python 2.2
* New Style Classes
  - the mro
  - super()

In [ ]:
class A: 
    def __init__(self): 
        
        print('A.__init__')
class B(A): 
    def __init__(self):  
        
        print('B.__init__')
class C(A): 
    def __init__(self): 
        
        print('C.__init__')
class D(B,C): 
    def __init__(self):
        
        print('D.__init__')

In [151]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [38]:
class A: 
    def __init__(self): 
        super(A,self).__init__()
        print('A.__init__')
class B(A): 
    def __init__(self):  
        super(B,self).__init__()
        print('B.__init__')
class C(A): 
    def __init__(self): 
        super(C,self).__init__()
        print('C.__init__')
class D(B,C): 
    def __init__(self):
        super(D,self).__init__()
        print('D.__init__')
        
D()

A.__init__
C.__init__
B.__init__
D.__init__


Where do we play our games? (10 minutes)

In [147]:
def my_decorator(klass):
    print(klass)
    return klass

@my_decorator
class MyClass: pass

<class '__main__.MyClass'>


In [2]:
def graffiti_decorator(klass):
    klass.tag = 'Kilroy was here'
    return klass

@graffiti_decorator
class MyClass: pass

MyClass.tag

'Kilroy was here'

* The metaclass - this is the class of the class 

In [155]:
class MyClass:pass
MyClass().__class__

__main__.MyClass

In [156]:
class MyMeta(type):pass
class MyClass(metaclass=MyMeta):pass
MyClass().__class__

__main__.MyClass

In [163]:
class MyMeta(type):
    def mro(cls):
        return super(MyMeta, cls).mro()
    
    def __new__(meta, name, bases, dct, **kwargs):
        return super(MyMeta, meta).__new__(meta, name, bases, dct, **kwargs)

In [164]:
class MyClass(metaclass=MyMeta):pass
MyClass().__class__

__main__.MyClass

In [6]:
class Negativity: pass

bool(Negativity)

True

In [1]:
class FalseMeta(type):
    
    def __bool__(self):
        return False
    
class Negativity(metaclass=FalseMeta): pass

bool(Negativity)

False

_I'm sure this will lead to no complications_

* class decorators

* \_\_subclasscheck\_\_ and \_\_instancecheck_\_ - Python 2.7

In [ ]:
def __instancecheck__(cls, instance):
    return super(mcls, cls).__instancecheck__(instance)
def __subclasscheck__(cls, subclass):
    return super(mcls, cls).__subclasscheck__(subclass)

In [34]:
class CheckCheckedMeta(type):
    def __instancecheck__(cls, instance):
        print(f'instance checked {instance}')
        return super(CheckCheckedMeta, cls).__instancecheck__(instance)
    def __subclasscheck__(cls, subclass):
        print(f'subclass checked {subclass}')
        return super(CheckCheckedMeta, cls).__subclasscheck__(subclass)

class CheckChecker(metaclass=CheckCheckedMeta): pass

In [35]:
isinstance(None,CheckChecker)

instance checked None


False

In [36]:
issubclass(object,CheckChecker)

subclass checked <class 'object'>


False

* \_\_init_subclass\_\_ - Python 3.6

In [ ]:
def __init_subclass__(cls, **kwargs):
    super().__init_subclass__(**kwargs)

In [38]:
class SizedContainer:
    def __init_subclass__(cls, **kwargs):
        cls.cls_field_count = len(cls.__dict__)
        super().__init_subclass__(**kwargs)

class Jar(SizedContainer):
    pickles = 18
    juice_oz = 12
          
Jar.cls_field_count

4

Let the games begin! (10 minutes)

* Turning ABCs into interfaces
  - stripping the non-abstract methods

In [2]:
import abc


class SloppyInterface(abc.ABC):
    @abc.abstractmethod
    def fetch_bread(self):pass
    @abc.abstractmethod
    def fetch_meat(self):pass
    
    def make_sandwich(self):
        return [self.fetch_bread(), 
                self.fetch_meat(), 
                self.fetch_bread()]

In [3]:
SloppyInterface.__dict__.keys()

dict_keys(['__module__', 'fetch_bread', 'fetch_meat', 'make_sandwich', '__dict__', '__weakref__', '__doc__', '__abstractmethods__', '_abc_impl'])

In [4]:
def force_interface(klass):
    for f, d in tuple(klass.__dict__.items()):
        if not hasattr(d,'__isabstractmethod__') \
        and f not in ('__dict__','__module__',
                      '__doc__','__weakref__',
                      '__abstractmethods__',
                      '_abc_impl'):
            delattr(klass, f) 
    return klass

@force_interface
class SloppyInterface(abc.ABC):
    @abc.abstractmethod
    def fetch_bread(self): pass
    @abc.abstractmethod
    def fetch_meat(self): pass
    
    def make_sandwich(self):
        return [self.fetch_bread(), 
                self.fetch_meat(), 
                self.fetch_bread()]      

In [5]:
SloppyInterface.make_sandwich()

AttributeError: type object 'SloppyInterface' has no attribute 'make_sandwich'

In [6]:
SloppyInterface.__dict__.keys()

dict_keys(['__module__', 'fetch_bread', 'fetch_meat', '__dict__', '__weakref__', '__doc__', '__abstractmethods__', '_abc_impl'])

In [14]:
class ForceInterface(abc.ABC):
  def __init_subclass__(cls):
    super().__init_subclass__()
    for f, d in tuple(
            cls.__dict__.items()):
      if not hasattr(d,
            '__isabstractmethod__')\
        and f not in ('__dict__',
                    '__module__',
                    '__doc__',
                    '__weakref__',
                    '__abstractmethods__',
                    '_abc_impl'):
                delattr(cls, f) 
                
class SloppyInterface(ForceInterface):
    @abc.abstractmethod
    def fetch_bread(self): pass
    @abc.abstractmethod
    def fetch_meat(self): pass
    
    def make_sandwich(self):
        return [self.fetch_bread(), 
                self.fetch_meat(), 
                self.fetch_bread()]

SloppyInterface.__dict__.keys()

dict_keys(['__module__', 'fetch_bread', 'fetch_meat', '__doc__', '__abstractmethods__', '_abc_impl'])

__Real Talk:__ ABCs are good as they are, this is not likely to be something you ever want to do, though you might use a similar technique to help document.  

In [8]:
class RealSandwich(SloppyInterface):
    def __init_subclass__(cls, **kwargs): 
        pass
class BeefBrisket(RealSandwich):
    def fetch_bread(self): 
        return "1 Slice Wonder Bread"
    def fetch_meat(self): 
        return "Beef Brisket"
    def make_sandwich(self):
        return [self.fetch_bread(), 
                self.fetch_meat(), 
                self.fetch_bread()]    

In [9]:
BeefBrisket().make_sandwich()


['1 Slice Wonder Bread', 'Beef Brisket', '1 Slice Wonder Bread']

# FakeInterface 1

* The FakeInterface
  - an interface with no inheritance at all
  - \_\_subclasshook\_\_ to pretend inheritance

In [4]:
class FakeInterfaceMeta(type):
    def __instancecheck__(cls, instance):
        return cls.__subclasscheck__(instance.__class__)
    def __subclasscheck__(cls, subclass):
        if not hasattr(subclass, 'type_flag'):
            return super(FakeInterfaceMeta, cls).__subclasscheck__(subclass)
        return cls.type_flag == subclass.type_flag

class FakeInterface(metaclass=FakeInterfaceMeta): pass
    
class Stock(FakeInterface):
    type_flag = 'STOCK'

class PreferredStock(FakeInterface):
    type_flag = 'PREFERRED'
    

In [104]:
class IbmC: type_flag = 'STOCK'
class IbmP: type_flag = 'PREFERRED'

In [105]:
issubclass(IbmC, PreferredStock)

False

In [106]:
issubclass(IbmC, Stock)

True

In [6]:
issubclass(object, Stock)

False

In [107]:
IbmC.__bases__, Stock.__bases__

((object,), (object,))

In [108]:
isinstance(IbmC(),Stock)

True

__Real Talk:__  This alone is not enough to justify using isinstance/isubclass, but if you are already using it for another reason you can use this to enable special rules.

# FakeInterface 2

* The FakeInterface
  - an interface with no inheritance at all
  - let’s assume we’re reading in data that used to be classes but is now just dicts.

In [ ]:
class FakeInterfaceMeta(type):
    def __instancecheck__(cls, instance):
        if not hasattr(instance, 'get'):
            return super(FakeInterfaceMeta, cls).__instancecheck__(instance)
        return cls.type_flag == instance.get('type_flag', None)

class FakeInterface(metaclass=FakeInterfaceMeta): pass

In [31]:
class Stock(FakeInterface):
    type_flag = 'STOCK'

class PreferredStock(FakeInterface):
    type_flag = 'PREFERRED'
    

In [32]:
ibmc = {'type_flag': 'STOCK', 'SYM': 'IBM', 'Price': 135.47} 
ibmp = {'type_flag': 'PREFERRED', 'SYM': 'IBM-A', 'Price': None}

In [33]:
isinstance(ibmc, Stock)

True

In [34]:
isinstance(ibmc, PreferredStock)

False

In [35]:
isinstance(ibmp, PreferredStock)

True

In [36]:
issubclass(dict, Stock)

False

In [ ]:
class FakeInterfaceMeta(type):
    def __instancecheck__(cls, instance):
        if not hasattr(instance, 'get'):
            return super(FakeInterfaceMeta, cls).__instancecheck__(instance)
        return cls.type_flag == instance.get('type_flag', None)

class FakeInterface(metaclass=FakeInterfaceMeta): pass

__Real Talk:__  This alone is not enough to justify using isinstance/isubclass, but if you are already using it for another reason you can use this to enable special rules.

* Exempting a few methods

In [1]:
class MetaUtil(type):
    def mro(cls):
        mro = type.mro(cls)
        if UtilityClass in mro:
            mro.remove(UtilityClass)
        mro = [mro[0], UtilityClass]+\
            mro[1:]
        return mro

class UtilityClass:
    @property
    def name(self): 
        return self.__class__.__name__
    
class Vehicle(metaclass=MetaUtil):
    pass
    
class Car(Vehicle):
    name = 'Automobile'
    def start(self):
        print('Turn key')

class Taxi(Car): pass

In [2]:
Taxi.__mro__

(__main__.Taxi, __main__.UtilityClass, __main__.Car, __main__.Vehicle, object)

In [3]:
Taxi().start()

Turn key


In [4]:
Taxi().name

'Taxi'

In [5]:
Car().name

'Automobile'

* making some methods that can't be written in subclasses

In [17]:
class MetaUtil(type):
    def mro(cls):
        mro = type.mro(cls)
        if UtilityClass in mro:
            mro.remove(UtilityClass)
        mro = [UtilityClass]+\
            mro
        return mro

class UtilityClass:
    @property
    def name(self): 
        return self.__class__.__name__
   
class Vehicle(metaclass=MetaUtil):
    pass  

In [ ]:
class Car(Vehicle):
    def start(self):
        print('Turn key')

In [15]:
Car().name

'Car'

In [16]:
Car().start()

Turn key


In [11]:
class Taxi(Car):
    name = 'Ride Share'

In [13]:
Taxi().start()

Turn key


In [14]:
Taxi().name

'Taxi'

In [12]:
Taxi.__mro__

(__main__.UtilityClass, __main__.Taxi, __main__.Car, __main__.Vehicle, object)

In [ ]:
class Vehicle(metaclass=MetaUtil):
    pass    

__Real Talk__:  This might actually be the safest to use thing in here.

* The SimpleObject - a base class with reduced capabilities
  - altering the MRO to control the behavior of super() 
  - disable __new__ and other hooks you choose not to support

In [25]:
class SimpleBase(type):
    def mro(cls):
        ret = [cls]
        while len(cls.__bases__):
            cls = cls.__bases__[0]
            ret.append(cls)
        return ret
        
class SimpleObject(metaclass=SimpleBase): 
    pass

In [26]:
class A(SimpleObject): pass
class B(A): pass
class C(A): pass
class D(B,C): 
    def __init__(self):
        print(self)

In [59]:
class A(SimpleObject): 
    def __init__(self): 
        super(A,self).__init__()
        print('A.__init__')
class B(A): 
    def __init__(self):  
        super(B,self).__init__()
        print('B.__init__')
class C(A): 
    def __init__(self): 
        super(C,self).__init__()
        print('C.__init__')
class D(B,C): 
    def __init__(self):
        super(D,self).__init__()
        print('D.__init__')
        
D()

A.__init__
B.__init__
D.__init__


__Real Talk:__ Again the point is we can pick and choose some features, not which features we're picking.  I might particularly suggest removing bases.

In [2]:
class SimpleBase(type):
    def __new__(meta, name, bases, dct, **kwargs):
        return super(SimpleBase, meta).__new__(meta, name, bases[0:1], dct, **kwargs)
    
class SimpleObject(metaclass=SimpleBase): 
    pass

In [3]:
class A(SimpleObject): pass
class B(A): pass
class C(A): pass
class D(B,C): 
    def __init__(self):
        print(self)

In [4]:
class A(SimpleObject): 
    def __init__(self): 
        super(A,self).__init__()
        print('A.__init__')
class B(A): 
    def __init__(self):  
        super(B,self).__init__()
        print('B.__init__')
class C(A): 
    def __init__(self): 
        super(C,self).__init__()
        print('C.__init__')
class D(B,C): 
    def __init__(self):
        super(D,self).__init__()
        print('D.__init__')
        
D()

A.__init__
B.__init__
D.__init__


In [36]:
for klass in [SimpleObject,A,B,C,D]:
    mro = '\t'.join('{:^10}'.format(k.__name__ if k in klass.__mro__ else '-') for k in [D,C,B,A,SimpleObject,object])
    print(f'{klass.__name__:12}: \t{mro}')

SimpleObject: 	    -     	    -     	    -     	    -     	SimpleObject	  object  
A           : 	    -     	    -     	    -     	    A     	    -     	  object  
B           : 	    -     	    -     	    B     	    A     	    -     	  object  
C           : 	    -     	    C     	    -     	    A     	    -     	  object  
D           : 	    D     	    -     	    B     	    A     	    -     	  object  


In [96]:
D()

# This approach breaks ABCs 

In [74]:
class interface_metaclass(abc.ABCMeta):
    def mro(cls):
        return  tuple(base for base in 
                      super(interface_metaclass, cls).mro() 
                      if not getattr(base,'__abstractmethods__',0))

In [75]:
class RetTrueABC(abc.ABC):
    @abc.abstractmethod
    def return_true(self):
        pass    
    def return_value(self): 
        return self.return_true()

class RetTrueInterface(RetTrueABC, 
                       metaclass=interface_metaclass):
    pass

In [76]:
class AsAbc(RetTrueABC):
    def return_true(self):
        return "True"
    
class AsInterface(RetTrueInterface):
    def return_true(self):
        return "TRUTH (out there)"
    

In [77]:
AsAbc().return_value()

'True'

In [78]:
AsInterface().return_true()

'TRUTH (out there)'

In [79]:
AsInterface().return_value()

AttributeError: 'AsInterface' object has no attribute 'return_value'

In [83]:
class BadInterface(RetTrueInterface):
    def somethingElse(self):pass
BadInterface()

TypeError: Can't instantiate abstract class BadInterface with abstract methods return_true

In [66]:
class SimpleBase(type):
    def __get_attr(self, field):
        print(f'SimpleBase {self} {field}')
    def mro(cls):
        if len(cls.__bases__):
            cls.__main_base__ = cls.__bases__[0]
        else:
            cls.__main_base__ = object
        print(f'{cls.__dict__}')
        print(cls.__dict__["__main_base__"])
        print(getattr(cls,"__main_base__"))
#        print(f'{cls.__name__} - {cls.__bases__} {getattr(cls,"__main_base__")}')
    
        ret = [cls]
        while hasattr(cls,'__main_base__'):
            cls = cls.__main_base__
            ret.append(cls)
        return ret
    
class SimpleObject(metaclass=SimpleBase): pass
class A(SimpleObject): pass
class B(A): pass
class C(A): pass
class D(B,C): pass

{'__module__': '__main__', '__dict__': <attribute '__dict__' of 'SimpleObject' objects>, '__weakref__': <attribute '__weakref__' of 'SimpleObject' objects>, '__main_base__': <class 'object'>}
<class 'object'>


AttributeError: type object 'SimpleObject' has no attribute '__main_base__'